In [1]:
!ls

Test-Ganomaly.ipynb  data  ganomaly  load_oasis3_data.ipynb


In [1]:
!pip install -r ganomaly/requirements.txt

     |████████████████████████████████| 157 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 430 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 7.6 MB/s  eta 0:00:01
     |████████████████████████████████| 278 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 13.1 MB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 226 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 6.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 

      Successfully uninstalled kiwisolver-1.3.1
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.3
    Uninstalling matplotlib-3.3.3:
      Successfully uninstalled matplotlib-3.3.3
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 20.0.0
    Uninstalling pyzmq-20.0.0:
      Successfully uninstalled pyzmq-20.0.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.2
    Uninstalling urllib3-1.26.2:
      Successfull

In [2]:
!pip install mkl-fft

     |████████████████████████████████| 232 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 396 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 11.2 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 737.3 MB 3.6 kB/s  eta 0:00:01     |██████████████▎                 | 329.2 MB 11.1 MB/s eta 0:00:37     |██████████████████████████████▊ | 708.3 MB 12.8 MB/s eta 0:00:03
     |████████████████████████████████| 425 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 919 kB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 7.1 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

mkl 2021.1.1 req

In [41]:
import os
import glob
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms


In [14]:
from PIL import Image
import cv2
import os.path
import numpy as np
import pickle
from typing import Any, Callable, Optional, Tuple

from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive

In [4]:
print(torch.__version__)

1.2.0


In [7]:
import torchvision
torchvision.__version__

'0.4.0'

In [9]:
dir(ImageFolder)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_find_classes',
 '_format_transform_repr',
 '_repr_indent',
 'extra_repr']

In [77]:
train_image_number = 140 # Image slice from MRI scans to use for training data

base_folder = '/pyt_oasis3'
non_ad_jpg_folder = '/tmp/data/t1w_non_ad_jpgs/' + str(train_image_number) + '/'
ad_jpg_folder = '/tmp/data/t1w_ad_jpgs/'

class Oasis3_train_normal(VisionDataset):
    def __init__(self,
            root: str,
            train: bool = True, # Train or test dataset
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None) -> None:
        super(Oasis3_train_normal, self).__init__(root, transform=transform, target_transform=target_transform)
        self.train = train # Train or test set
        
        self.data = []
        self.targets = []
        
        for file_path in glob.glob(non_ad_jpg_folder+'**.jpg'):
            with open (file_path, 'rb') as f:
                img = cv2.imread(file_path)
                # todo: Resize all images to 176, 256, 3
                self.data.append(img)
                self.targets.extend(str(0)) # based on the jpg_folder in for loop
#        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
#        self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        img, target = self.data[index], self.targets[index]
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

In [78]:
a=Oasis3_train_normal(base_folder, False)

In [81]:
a.data[0].shape

(176, 256, 3)

In [86]:
# Check if all images loaded into the class are of same size so that they can be reshaped
for i in range(0, len(a.data)):
    if a.data[i].shape != (176, 256, 3):
        print(f"index: {i}, shape: {a.data[i].shape}")
        print("--")

index: 5, shape: (176, 240, 3)
--
index: 7, shape: (176, 240, 3)
--
index: 10, shape: (176, 240, 3)
--
index: 12, shape: (176, 240, 3)
--
index: 14, shape: (176, 240, 3)
--
index: 15, shape: (176, 240, 3)
--
index: 16, shape: (176, 240, 3)
--
index: 24, shape: (176, 240, 3)
--
index: 35, shape: (176, 240, 3)
--
index: 36, shape: (176, 240, 3)
--
index: 38, shape: (176, 240, 3)
--
index: 43, shape: (176, 240, 3)
--
index: 44, shape: (176, 240, 3)
--
index: 61, shape: (176, 240, 3)
--
index: 68, shape: (176, 240, 3)
--
index: 69, shape: (176, 240, 3)
--
index: 70, shape: (176, 240, 3)
--
index: 74, shape: (176, 240, 3)
--
index: 77, shape: (176, 240, 3)
--
index: 78, shape: (176, 240, 3)
--
index: 82, shape: (176, 240, 3)
--
index: 85, shape: (176, 240, 3)
--
index: 86, shape: (176, 240, 3)
--
index: 87, shape: (176, 240, 3)
--
index: 93, shape: (176, 240, 3)
--
index: 99, shape: (176, 240, 3)
--
index: 102, shape: (176, 240, 3)
--
index: 106, shape: (176, 240, 3)
--
index: 107, shape: (

In [83]:
foo = np.vstack(a.data).reshape(-1, 176, 256, 3)

ValueError: all the input array dimensions except for the concatenation axis must match exactly